## Question 3: Cultural Arts Organizations course types and number of instructional hours per year correlation with ELA and math performance

Author: Thomas

Updates:
* Last updated on March 26, 2020
* Updated on March 25, 2020
* Updated on March 24, 2020
* Updated on March 23, 2020
* Updated on March 21, 2020

**Hypothesis**: If art programs provided by Cultural Arts Organizations affect student performance, then schools with stronger indicators of instructional hours will have significantly different math and ela state test scores than schools with weaker indicators of instructional hours.

**Null hypothesis**: If art programs provided by Cultural Arts Organizations do not affect student performance, then schools with stronger indicators of instructional hours will not have significantly different math and ELA state test scores than schools with weaker indicators of instructional hours.

**Indicators of instructional hours** will be: (a) number of art course types and (b) self-report of instructional hours. 

**Math/ELA performance** will be measured by pass rate (proportion of students earning a score of 3 or 4).

## Data

### School Test Results Data
2013-2019 School Test Results Page
https://infohub.nyced.org/reports/academics/test-results

2013-2019 ELA Test Results: 
https://infohub.nyced.org/docs/default-source/default-document-library/school-ela-results-2013-2019-(public).xlsx
2013-2019 Math Test Results: 
https://infohub.nyced.org/docs/default-source/default-document-library/school-math-results-2013-2019-(public).xlsx

### Arts Survey Data
2016-2017 Arts Survey Data
https://data.cityofnewyork.us/Education/2016-2017-Arts-Survey-data/f33j-ecpr

2017-2018 Arts Survey Data
https://data.cityofnewyork.us/Education/2017-2018-Arts-Data-Report/d9fr-a56v

2018-2019 Arts Data Survey
https://data.cityofnewyork.us/Education/2018-2019-Arts-Data-Survey/5cxm-c27f

In [1]:
# import dependencies

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import json


## Dataset 1: Math and ELA Tests Data

In [2]:
# CSV Files
math_results = 'data/2013-2019_school_math_results.csv'
ela_results = 'data/2013-2019_school_ela_results.csv'

In [3]:
# read csv file into dataframe
math_df = pd.read_csv(math_results, encoding='utf-8', low_memory=False)

# clean up math dataframe

keep_cols = ['DBN','Year','Grade','Number Tested','# Level 3+4', f'% Level 3+4']
keep_grades = ['3', '4','5','6']

# drop unneeded columns
math_df = math_df.drop(labels='Unnamed: 0', axis=1)[math_df.Year >= 2017].reset_index().drop(labels='index', axis=1)[keep_cols]
# remove all rows matching `All Grades` grade
math_df = math_df[ math_df.Grade != 'All Grades' ]
# set `Grade` to int type
math_df.Grade = math_df.Grade.astype(int)
# keep only grades from 3 to 6
math_df = math_df[ (math_df.Grade >= 3) & (math_df.Grade <= 6) ]
# add test column
math_df['Test'] = 'Math'

print(math_df.shape)
math_df.head(1)


(8278, 7)


,DBN,Year,Grade,Number Tested,# Level 3+4,% Level 3+4,Test
0,01M015,2017,3,29,12,41.37931061,Math


In [4]:
# read csv file into dataframe
ela_df = pd.read_csv(ela_results, encoding='utf-8', low_memory=False)

# clean up ela dataframe

# drop unneeded columns
ela_df = ela_df.drop(labels='Unnamed: 0', axis=1)[ela_df.Year >= 2017].reset_index().drop(labels='index', axis=1)[keep_cols]
# remove all rows matching `All Grades` grade
ela_df = ela_df[ ela_df.Grade != 'All Grades' ]
# set `Grade` to int type
ela_df.Grade = ela_df.Grade.astype(int)
# keep only grades from 3 to 6
ela_df = ela_df[ (ela_df.Grade >= 3) & (ela_df.Grade <= 6) ]
# add test column
ela_df['Test'] = 'ELA'

print(ela_df.shape)
ela_df.head(1)

(8276, 7)


,DBN,Year,Grade,Number Tested,# Level 3+4,% Level 3+4,Test
0,01M015,2017,3,27,9,33.33333206,ELA


In [5]:
# concatenate both dataframes into one
tests_df = pd.concat([math_df, ela_df])

# clean up dataframe

# cast values to numeric types and force non-numeric to NaN
tests_df['# Level 3+4'] = pd.to_numeric(tests_df['# Level 3+4'], errors='coerce')
tests_df['% Level 3+4'] = pd.to_numeric(tests_df['% Level 3+4'], errors='coerce')

# drop any rows with any NaN values
tests_df = tests_df.dropna(how='any')

# cast to integer
tests_df['# Level 3+4'] = tests_df['# Level 3+4'].astype(int)

# reset index and drop extra column
tests_df = tests_df.reset_index().drop(labels='index', axis=1)

print(tests_df.shape)
tests_df.head(1)

(16520, 7)


,DBN,Year,Grade,Number Tested,# Level 3+4,% Level 3+4,Test
0,01M015,2017,3,29,12,41.379311,Math


## Dataset 2: Art Surveys Data

In [6]:
# CSV files
arts_2017 = 'data/2016-2017_Arts_Survey_Data.csv'
arts_2018 = 'data/2017-2018_Arts_Survey_Data.csv'
arts_2019 = 'data/2018-2019_Arts_Survey_Data.csv'

### Load and clean data for 2017 Arts Survey

In [7]:
arts_2017_df = pd.read_csv(arts_2017, encoding='utf-8', low_memory=False)
arts_2017_df = arts_2017_df.rename(columns={ 'Q0_DBN': 'DBN' })
arts_2017_df['Year'] = 2017

arts_2017_grade_3_dance_cols          = [  col for col in arts_2017_df.columns if all(ele in col for ele in [ 'Q15', 'R1' ]) ]
arts_2017_grade_3_music_cols          = [  col for col in arts_2017_df.columns if all(ele in col for ele in [ 'Q15', 'R2' ]) ]
arts_2017_grade_3_theater_cols        = [  col for col in arts_2017_df.columns if all(ele in col for ele in [ 'Q15', 'R3' ]) ]
arts_2017_grade_3_visual_cols         = [  col for col in arts_2017_df.columns if all(ele in col for ele in [ 'Q15', 'R4' ]) ]

arts_2017_grade_3_df = pd.DataFrame()
arts_2017_grade_3_df['Dance']         = arts_2017_df[arts_2017_grade_3_dance_cols].fillna(0).sum(axis=1)
arts_2017_grade_3_df['Music']         = arts_2017_df[arts_2017_grade_3_music_cols].fillna(0).sum(axis=1)
arts_2017_grade_3_df['Theater']       = arts_2017_df[arts_2017_grade_3_theater_cols].fillna(0).sum(axis=1)
arts_2017_grade_3_df['Visual Arts']   = arts_2017_df[arts_2017_grade_3_visual_cols].fillna(0).sum(axis=1)
arts_2017_grade_3_df[['DBN','Year']]  = arts_2017_df[['DBN','Year']]
arts_2017_grade_3_df['Grade']         = 3

arts_2017_grade_4_dance_cols          = [  col for col in arts_2017_df.columns if all(ele in col for ele in [ 'Q16', 'R1' ]) ]
arts_2017_grade_4_music_cols          = [  col for col in arts_2017_df.columns if all(ele in col for ele in [ 'Q16', 'R2' ]) ]
arts_2017_grade_4_theater_cols        = [  col for col in arts_2017_df.columns if all(ele in col for ele in [ 'Q16', 'R3' ]) ]
arts_2017_grade_4_visual_cols         = [  col for col in arts_2017_df.columns if all(ele in col for ele in [ 'Q16', 'R4' ]) ]

arts_2017_grade_4_df = pd.DataFrame()
arts_2017_grade_4_df['Dance']         = arts_2017_df[arts_2017_grade_4_dance_cols].fillna(0).sum(axis=1)
arts_2017_grade_4_df['Music']         = arts_2017_df[arts_2017_grade_4_music_cols].fillna(0).sum(axis=1)
arts_2017_grade_4_df['Theater']       = arts_2017_df[arts_2017_grade_4_theater_cols].fillna(0).sum(axis=1)
arts_2017_grade_4_df['Visual Arts']   = arts_2017_df[arts_2017_grade_4_visual_cols].fillna(0).sum(axis=1)
arts_2017_grade_4_df[['DBN','Year']]  = arts_2017_df[['DBN','Year']]
arts_2017_grade_4_df['Grade']         = 4

arts_2017_grade_5_dance_cols          = [  col for col in arts_2017_df.columns if all(ele in col for ele in [ 'Q17', 'R1' ]) ]
arts_2017_grade_5_music_cols          = [  col for col in arts_2017_df.columns if all(ele in col for ele in [ 'Q17', 'R2' ]) ]
arts_2017_grade_5_theater_cols        = [  col for col in arts_2017_df.columns if all(ele in col for ele in [ 'Q17', 'R3' ]) ]
arts_2017_grade_5_visual_cols         = [  col for col in arts_2017_df.columns if all(ele in col for ele in [ 'Q17', 'R4' ]) ]

arts_2017_grade_5_df = pd.DataFrame()
arts_2017_grade_5_df['Dance']         = arts_2017_df[arts_2017_grade_5_dance_cols].fillna(0).sum(axis=1)
arts_2017_grade_5_df['Music']         = arts_2017_df[arts_2017_grade_5_music_cols].fillna(0).sum(axis=1)
arts_2017_grade_5_df['Theater']       = arts_2017_df[arts_2017_grade_5_theater_cols].fillna(0).sum(axis=1)
arts_2017_grade_5_df['Visual Arts']   = arts_2017_df[arts_2017_grade_5_visual_cols].fillna(0).sum(axis=1)
arts_2017_grade_5_df[['DBN','Year']]  = arts_2017_df[['DBN','Year']]
arts_2017_grade_5_df['Grade']         = 5

arts_2017_grade_6_dance_cols          = [  col for col in arts_2017_df.columns if all(ele in col for ele in [ 'Q19', 'R1' ]) ]
arts_2017_grade_6_music_cols          = [  col for col in arts_2017_df.columns if all(ele in col for ele in [ 'Q19', 'R2' ]) ]
arts_2017_grade_6_theater_cols        = [  col for col in arts_2017_df.columns if all(ele in col for ele in [ 'Q19', 'R3' ]) ]
arts_2017_grade_6_visual_cols         = [  col for col in arts_2017_df.columns if all(ele in col for ele in [ 'Q19', 'R4' ]) ]

arts_2017_grade_6_df = pd.DataFrame()
arts_2017_grade_6_df['Dance']         = arts_2017_df[arts_2017_grade_6_dance_cols].fillna(0).sum(axis=1)
arts_2017_grade_6_df['Music']         = arts_2017_df[arts_2017_grade_6_music_cols].fillna(0).sum(axis=1)
arts_2017_grade_6_df['Theater']       = arts_2017_df[arts_2017_grade_6_theater_cols].fillna(0).sum(axis=1)
arts_2017_grade_6_df['Visual Arts']   = arts_2017_df[arts_2017_grade_6_visual_cols].fillna(0).sum(axis=1)
arts_2017_grade_6_df[['DBN','Year']]  = arts_2017_df[['DBN','Year']]
arts_2017_grade_6_df['Grade']         = 6

arts_2017_grades = pd.concat([arts_2017_grade_3_df, arts_2017_grade_4_df, arts_2017_grade_5_df, arts_2017_grade_6_df])
arts_2017_grades = arts_2017_grades[arts_2017_grades.columns.tolist()[-3:] + arts_2017_grades.columns.tolist()[:-3]]
print(arts_2017_grades.shape)
arts_2017_grades.head(1)


(5904, 7)


,DBN,Year,Grade,Dance,Music,Theater,Visual Arts
0,01M015,2017,3,36.0,56.0,10.0,20.0


In [8]:
arts_2018_df = pd.read_csv(arts_2018, encoding='utf-8', low_memory=False)
arts_2018_df = arts_2018_df.rename(columns={ 'Q0_DBN': 'DBN' })
arts_2018_df['Year'] = 2018

arts_2018_grade_3_dance_cols          = [  col for col in arts_2018_df.columns if all(ele in col for ele in [ 'Q15', 'R1' ]) ]
arts_2018_grade_3_music_cols          = [  col for col in arts_2018_df.columns if all(ele in col for ele in [ 'Q15', 'R2' ]) ]
arts_2018_grade_3_theater_cols        = [  col for col in arts_2018_df.columns if all(ele in col for ele in [ 'Q15', 'R3' ]) ]
arts_2018_grade_3_visual_cols         = [  col for col in arts_2018_df.columns if all(ele in col for ele in [ 'Q15', 'R4' ]) ]

arts_2018_grade_3_df = pd.DataFrame()
arts_2018_grade_3_df['Dance']         = arts_2018_df[arts_2018_grade_3_dance_cols].fillna(0).sum(axis=1)
arts_2018_grade_3_df['Music']         = arts_2018_df[arts_2018_grade_3_music_cols].fillna(0).sum(axis=1)
arts_2018_grade_3_df['Theater']       = arts_2018_df[arts_2018_grade_3_theater_cols].fillna(0).sum(axis=1)
arts_2018_grade_3_df['Visual Arts']   = arts_2018_df[arts_2018_grade_3_visual_cols].fillna(0).sum(axis=1)
arts_2018_grade_3_df[['DBN','Year']]  = arts_2018_df[['DBN','Year']]
arts_2018_grade_3_df['Grade']         = 3

arts_2018_grade_4_dance_cols          = [  col for col in arts_2018_df.columns if all(ele in col for ele in [ 'Q16', 'R1' ]) ]
arts_2018_grade_4_music_cols          = [  col for col in arts_2018_df.columns if all(ele in col for ele in [ 'Q16', 'R2' ]) ]
arts_2018_grade_4_theater_cols        = [  col for col in arts_2018_df.columns if all(ele in col for ele in [ 'Q16', 'R3' ]) ]
arts_2018_grade_4_visual_cols         = [  col for col in arts_2018_df.columns if all(ele in col for ele in [ 'Q16', 'R4' ]) ]

arts_2018_grade_4_df = pd.DataFrame()
arts_2018_grade_4_df['Dance']         = arts_2018_df[arts_2018_grade_4_dance_cols].fillna(0).sum(axis=1)
arts_2018_grade_4_df['Music']         = arts_2018_df[arts_2018_grade_4_music_cols].fillna(0).sum(axis=1)
arts_2018_grade_4_df['Theater']       = arts_2018_df[arts_2018_grade_4_theater_cols].fillna(0).sum(axis=1)
arts_2018_grade_4_df['Visual Arts']   = arts_2018_df[arts_2018_grade_4_visual_cols].fillna(0).sum(axis=1)
arts_2018_grade_4_df[['DBN','Year']]  = arts_2018_df[['DBN','Year']]
arts_2018_grade_4_df['Grade']         = 4

arts_2018_grade_5_dance_cols          = [  col for col in arts_2018_df.columns if all(ele in col for ele in [ 'Q17', 'R1' ]) ]
arts_2018_grade_5_music_cols          = [  col for col in arts_2018_df.columns if all(ele in col for ele in [ 'Q17', 'R2' ]) ]
arts_2018_grade_5_theater_cols        = [  col for col in arts_2018_df.columns if all(ele in col for ele in [ 'Q17', 'R3' ]) ]
arts_2018_grade_5_visual_cols         = [  col for col in arts_2018_df.columns if all(ele in col for ele in [ 'Q17', 'R4' ]) ]

arts_2018_grade_5_df = pd.DataFrame()
arts_2018_grade_5_df['Dance']         = arts_2018_df[arts_2018_grade_5_dance_cols].fillna(0).sum(axis=1)
arts_2018_grade_5_df['Music']         = arts_2018_df[arts_2018_grade_5_music_cols].fillna(0).sum(axis=1)
arts_2018_grade_5_df['Theater']       = arts_2018_df[arts_2018_grade_5_theater_cols].fillna(0).sum(axis=1)
arts_2018_grade_5_df['Visual Arts']   = arts_2018_df[arts_2018_grade_5_visual_cols].fillna(0).sum(axis=1)
arts_2018_grade_5_df[['DBN','Year']]  = arts_2018_df[['DBN','Year']]
arts_2018_grade_5_df['Grade']         = 5

arts_2018_grade_6_dance_cols          = [  col for col in arts_2018_df.columns if all(ele in col for ele in [ 'Q19', 'R1' ]) ]
arts_2018_grade_6_music_cols          = [  col for col in arts_2018_df.columns if all(ele in col for ele in [ 'Q19', 'R2' ]) ]
arts_2018_grade_6_theater_cols        = [  col for col in arts_2018_df.columns if all(ele in col for ele in [ 'Q19', 'R3' ]) ]
arts_2018_grade_6_visual_cols         = [  col for col in arts_2018_df.columns if all(ele in col for ele in [ 'Q19', 'R4' ]) ]

arts_2018_grade_6_df = pd.DataFrame()
arts_2018_grade_6_df['Dance']         = arts_2018_df[arts_2018_grade_6_dance_cols].fillna(0).sum(axis=1)
arts_2018_grade_6_df['Music']         = arts_2018_df[arts_2018_grade_6_music_cols].fillna(0).sum(axis=1)
arts_2018_grade_6_df['Theater']       = arts_2018_df[arts_2018_grade_6_theater_cols].fillna(0).sum(axis=1)
arts_2018_grade_6_df['Visual Arts']   = arts_2018_df[arts_2018_grade_6_visual_cols].fillna(0).sum(axis=1)
arts_2018_grade_6_df[['DBN','Year']]  = arts_2018_df[['DBN','Year']]
arts_2018_grade_6_df['Grade']         = 6

arts_2018_grades = pd.concat([arts_2018_grade_3_df, arts_2018_grade_4_df, arts_2018_grade_5_df, arts_2018_grade_6_df])
arts_2018_grades = arts_2018_grades[arts_2018_grades.columns.tolist()[-3:] + arts_2018_grades.columns.tolist()[:-3]]
print(arts_2018_grades.shape)
arts_2018_grades.head(1)


(5964, 7)


,DBN,Year,Grade,Dance,Music,Theater,Visual Arts
0,01M015,2018,3,0.0,60.0,20.0,0.0


In [9]:
arts_2019_df = pd.read_csv(arts_2019, encoding='utf-8', low_memory=False)
arts_2019_df = arts_2019_df.rename(columns={ 'Q0_DBN': 'DBN' })
arts_2019_df['Year'] = 2019

arts_2019_grade_3_dance_cols          = [  col for col in arts_2019_df.columns if all(ele in col for ele in [ 'Q15', 'R1' ]) ]
arts_2019_grade_3_music_cols          = [  col for col in arts_2019_df.columns if all(ele in col for ele in [ 'Q15', 'R2' ]) ]
arts_2019_grade_3_theater_cols        = [  col for col in arts_2019_df.columns if all(ele in col for ele in [ 'Q15', 'R3' ]) ]
arts_2019_grade_3_visual_cols         = [  col for col in arts_2019_df.columns if all(ele in col for ele in [ 'Q15', 'R4' ]) ]

arts_2019_grade_3_df = pd.DataFrame()
arts_2019_grade_3_df['Dance']         = arts_2019_df[arts_2019_grade_3_dance_cols].fillna(0).sum(axis=1)
arts_2019_grade_3_df['Music']         = arts_2019_df[arts_2019_grade_3_music_cols].fillna(0).sum(axis=1)
arts_2019_grade_3_df['Theater']       = arts_2019_df[arts_2019_grade_3_theater_cols].fillna(0).sum(axis=1)
arts_2019_grade_3_df['Visual Arts']   = arts_2019_df[arts_2019_grade_3_visual_cols].fillna(0).sum(axis=1)
arts_2019_grade_3_df[['DBN','Year']]  = arts_2019_df[['DBN','Year']]
arts_2019_grade_3_df['Grade']         = 3

arts_2019_grade_4_dance_cols          = [  col for col in arts_2019_df.columns if all(ele in col for ele in [ 'Q16', 'R1' ]) ]
arts_2019_grade_4_music_cols          = [  col for col in arts_2019_df.columns if all(ele in col for ele in [ 'Q16', 'R2' ]) ]
arts_2019_grade_4_theater_cols        = [  col for col in arts_2019_df.columns if all(ele in col for ele in [ 'Q16', 'R3' ]) ]
arts_2019_grade_4_visual_cols         = [  col for col in arts_2019_df.columns if all(ele in col for ele in [ 'Q16', 'R4' ]) ]

arts_2019_grade_4_df = pd.DataFrame()
arts_2019_grade_4_df['Dance']         = arts_2019_df[arts_2019_grade_4_dance_cols].fillna(0).sum(axis=1)
arts_2019_grade_4_df['Music']         = arts_2019_df[arts_2019_grade_4_music_cols].fillna(0).sum(axis=1)
arts_2019_grade_4_df['Theater']       = arts_2019_df[arts_2019_grade_4_theater_cols].fillna(0).sum(axis=1)
arts_2019_grade_4_df['Visual Arts']   = arts_2019_df[arts_2019_grade_4_visual_cols].fillna(0).sum(axis=1)
arts_2019_grade_4_df[['DBN','Year']]  = arts_2019_df[['DBN','Year']]
arts_2019_grade_4_df['Grade']         = 4

arts_2019_grade_5_dance_cols          = [  col for col in arts_2019_df.columns if all(ele in col for ele in [ 'Q17', 'R1' ]) ]
arts_2019_grade_5_music_cols          = [  col for col in arts_2019_df.columns if all(ele in col for ele in [ 'Q17', 'R2' ]) ]
arts_2019_grade_5_theater_cols        = [  col for col in arts_2019_df.columns if all(ele in col for ele in [ 'Q17', 'R3' ]) ]
arts_2019_grade_5_visual_cols         = [  col for col in arts_2019_df.columns if all(ele in col for ele in [ 'Q17', 'R4' ]) ]

arts_2019_grade_5_df = pd.DataFrame()
arts_2019_grade_5_df['Dance']         = arts_2019_df[arts_2019_grade_5_dance_cols].fillna(0).sum(axis=1)
arts_2019_grade_5_df['Music']         = arts_2019_df[arts_2019_grade_5_music_cols].fillna(0).sum(axis=1)
arts_2019_grade_5_df['Theater']       = arts_2019_df[arts_2019_grade_5_theater_cols].fillna(0).sum(axis=1)
arts_2019_grade_5_df['Visual Arts']   = arts_2019_df[arts_2019_grade_5_visual_cols].fillna(0).sum(axis=1)
arts_2019_grade_5_df[['DBN','Year']]  = arts_2019_df[['DBN','Year']]
arts_2019_grade_5_df['Grade']         = 5

arts_2019_grade_6_dance_cols          = [  col for col in arts_2019_df.columns if all(ele in col for ele in [ 'Q19', 'R1' ]) ]
arts_2019_grade_6_music_cols          = [  col for col in arts_2019_df.columns if all(ele in col for ele in [ 'Q19', 'R2' ]) ]
arts_2019_grade_6_theater_cols        = [  col for col in arts_2019_df.columns if all(ele in col for ele in [ 'Q19', 'R3' ]) ]
arts_2019_grade_6_visual_cols         = [  col for col in arts_2019_df.columns if all(ele in col for ele in [ 'Q19', 'R4' ]) ]

arts_2019_grade_6_df = pd.DataFrame()
arts_2019_grade_6_df['Dance']         = arts_2019_df[arts_2019_grade_6_dance_cols].fillna(0).sum(axis=1)
arts_2019_grade_6_df['Music']         = arts_2019_df[arts_2019_grade_6_music_cols].fillna(0).sum(axis=1)
arts_2019_grade_6_df['Theater']       = arts_2019_df[arts_2019_grade_6_theater_cols].fillna(0).sum(axis=1)
arts_2019_grade_6_df['Visual Arts']   = arts_2019_df[arts_2019_grade_6_visual_cols].fillna(0).sum(axis=1)
arts_2019_grade_6_df[['DBN','Year']]  = arts_2019_df[['DBN','Year']]
arts_2019_grade_6_df['Grade']         = 6

arts_2019_grades = pd.concat([arts_2019_grade_3_df, arts_2019_grade_4_df, arts_2019_grade_5_df, arts_2019_grade_6_df])
arts_2019_grades = arts_2019_grades[arts_2019_grades.columns.tolist()[-3:] + arts_2019_grades.columns.tolist()[:-3]]
print(arts_2019_grades.shape)
arts_2019_grades.head(1)


(6184, 7)


,DBN,Year,Grade,Dance,Music,Theater,Visual Arts
0,01M015,2019,3,0.0,60.0,22.0,0.0


In [39]:
arts_df = pd.concat([arts_2017_grades, arts_2018_grades, arts_2019_grades])

# reset index and drop extra column
arts_df = arts_df.reset_index().drop(labels='index', axis=1)

# check dataframe
print(arts_df.shape)
arts_df.head()

(18052, 7)


,DBN,Year,Grade,Dance,Music,Theater,Visual Arts
0,01M015,2017,3,36.0,56.0,10.0,20.0
1,01M019,2017,3,208.0,48.0,0.0,40.0
2,01M020,2017,3,0.0,0.0,0.0,5.0
3,01M034,2017,3,0.0,4.0,0.0,0.0
4,01M515,2017,3,0.0,0.0,0.0,0.0


In [40]:
# make sure no duplicate indices exist
from collections import Counter
Counter(arts_df.index.duplicated(keep='first'))

Counter({False: 18052})

In [12]:
# merge test scores with art survey data
data_df = pd.merge(tests_df, arts_df, how='outer', on=['DBN','Year','Grade'])

# drop any schools with no passing data
data_df = data_df.dropna(subset=['% Level 3+4'])

print(data_df.shape)
data_df.head(2)

(16520, 11)


,DBN,Year,Grade,Number Tested,# Level 3+4,% Level 3+4,Test,Dance,Music,Theater,Visual Arts
0,01M015,2017,3,29.0,12.0,41.379311,Math,36.0,56.0,10.0,20.0
1,01M015,2017,3,27.0,9.0,33.333332,ELA,36.0,56.0,10.0,20.0


In [58]:
data_df.columns[3:]

Index(['Number Tested', '# Level 3+4', '% Level 3+4', 'Test', 'Dance', 'Music',
       'Theater', 'Visual Arts'],
      dtype='object')

In [63]:
# create new table for the merged data for analysis
schools = pd.DataFrame(columns = data_df.columns[3:].to_list())

schools = schools.drop(labels='Test', axis=1)

for col in schools.columns:
    if ('%' in col):
        schools[col] = data_df.groupby(['DBN','Year','Test'])[col].mean()
    else:
        schools[col] = data_df.groupby(['DBN','Year','Test'])[col].sum()
        
schools.head()

Number Tested  # Level 3+4  % Level 3+4  Dance  Music  \
DBN    Year Test                                                          
01M015 2017 ELA            67.0         24.0    35.407785   72.0  112.0   
            Math           69.0         31.0    46.444425   72.0  112.0   
       2018 ELA            67.0         45.0    67.203906    8.0  150.0   
            Math           67.0         39.0    59.382716    8.0  150.0   
       2019 ELA            70.0         42.0    61.382716    0.0  180.0   
...                         ...          ...          ...    ...    ...   
32K562 2017 Math          112.0          7.0     6.250000    0.0   54.0   
       2018 ELA           111.0         37.0    33.333332    0.0    2.0   
            Math          110.0         14.0    12.727273    0.0    2.0   
       2019 ELA           105.0         28.0    26.666666    0.0    0.0   
            Math          108.0         25.0    23.148148    0.0    0.0   

                  Theater  Visual Arts  
DBN    Year Test                        
01M015 2017 ELA      35.0         20.0  
            Math     35.0         20.0  
       2018 ELA      48.0          0.0  
            Math     48.0          0.0  
       2019 ELA      80.0          0.0  
...                   ...          ...  
32K562 2017 Math      0.0         54.0  
       2018 ELA       0.0          0.0  
            Math      0.0          0.0  
       2019 ELA       0.0          0.0  
            Math      0.0          0.0  

[6612 rows x 7 columns]

In [ ]:
# enrollment and attendance counts
attend1 = pd.read_csv('data/2015-2018_Daily_Attendance.csv', encoding='utf-8', low_memory=False)

# clean up data
attend1 = attend1.rename(columns={ 'School': 'DBN' })
attend1['Year'] = attend1['SchoolYear'].astype(str).map(lambda x: int(x[-4:]) )

# drop unneeded columns
attend1 = attend1.drop(labels='SchoolYear', axis=1)

# reorder columns
ro_cols = attend1.columns.tolist()[:1]+attend1.columns.tolist()[-1:]+attend1.columns.tolist()[:-1][1:]
attend1 = attend1[ro_cols]

print(attend1.shape)
attend1.head(1)